# HullBuilder testings

## Imports and setup

In [15]:
from pcgsepy.common.api_call import get_base_values, GameMode, toggle_gamemode
from pcgsepy.common.vecs import Vec, Orientation
from pcgsepy.lsystem.structure_maker import LLStructureMaker
from pcgsepy.setup_utils import setup_matplotlib, get_default_lsystem
from pcgsepy.structure import Structure, place_structure, place_blocks

In [16]:
setup_matplotlib(larger_fonts=False)

used_ll_blocks = [
    'MyObjectBuilder_CubeBlock_LargeBlockArmorCornerInv',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorCorner',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorSlope',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorBlock',
    'MyObjectBuilder_Gyro_LargeBlockGyro',
    'MyObjectBuilder_Reactor_LargeBlockSmallGenerator',
    'MyObjectBuilder_CargoContainer_LargeBlockSmallContainer',
    'MyObjectBuilder_Cockpit_OpenCockpitLarge',
    'MyObjectBuilder_Thrust_LargeBlockSmallThrust',
    'MyObjectBuilder_InteriorLight_SmallLight',
    'MyObjectBuilder_CubeBlock_Window1x1Slope',
    'MyObjectBuilder_CubeBlock_Window1x1Flat',
    'MyObjectBuilder_InteriorLight_LargeBlockLight_1corner'
]

lsystem = get_default_lsystem(used_ll_blocks=used_ll_blocks)

In [17]:
import numpy as np
import plotly.express as px

def interactive_plot(structure: Structure,
                     title: str) -> None:
    content = structure.as_grid_array()
    arr = np.nonzero(content)
    x, y, z = arr
    cs = [content[i, j, k] for i, j, k in zip(x, y, z)]
    ss = [structure._clean_label(structure.ks[v - 1]) for v in cs]
    fig = px.scatter_3d(x=x,
                        y=y,
                        z=z,
                        color=ss,
                        # color_discrete_map=_get_colour_mapping(ss),
                        labels={
                            'x': 'x',
                            'y': 'y',
                            'z': 'z',
                            'color': 'Block type'
                        },
                        title=title)

    fig.update_traces(marker=dict(size=4,
                                line=dict(width=3,
                                        color='DarkSlateGrey')),
                        selector=dict(mode='markers'))

    camera = dict(
        up=dict(x=0, y=0, z=1),
        center=dict(x=0, y=0, z=0),
        eye=dict(x=2, y=2, z=2)
        )

    fig.update_layout(scene=dict(aspectmode='data'),
                        scene_camera=camera,
                        paper_bgcolor='rgba(0,0,0,0)',
                        plot_bgcolor='rgba(0,0,0,0)')

    fig.show()

## Spaceship string

Define here the high-level spaceship string.

In [18]:
spaceship_string = 'cockpitcorridorsimple(2)[RotYcwXcorridorsimple(1)]thrusters'

## Spaceship creation

In [19]:
ml_string = lsystem.hl_solver.translator.transform(string=spaceship_string)
ll_solution = lsystem.ll_solver.solve(string=ml_string,
                                   iterations=1,
                                   strings_per_iteration=1,
                                   check_sat=False)[0]
base_position, orientation_forward, orientation_up = Vec.v3i(
            0, 0, 0), Orientation.FORWARD.value, Orientation.UP.value
structure = Structure(origin=base_position,
                      orientation_forward=orientation_forward,
                      orientation_up=orientation_up)
structure = LLStructureMaker(
    atoms_alphabet=lsystem.ll_solver.atoms_alphabet,
    position=base_position).fill_structure(structure=structure,
                                           string=ll_solution.string)
structure.sanify()
interactive_plot(structure=structure,
                 title='Original structure')

## Hullbuilder

In [20]:
from typing import Tuple
from pcgsepy.structure import Block

def vec_to_idx(v: Vec) -> Tuple[int, int, int]:
    return (v.x, v.y, v.z)

def idx_to_vec(idx: Tuple[int, int, int]) -> Vec:
    return Vec(x=idx[0], y=idx[1], z=idx[2])

def is_slope_block(block: Block) -> bool:
    return "Slope" in block.block_type or "Corner" in block.block_type

In [21]:
from itertools import combinations
from scipy.spatial import ConvexHull, Delaunay
from scipy.ndimage import grey_erosion, binary_erosion
import numpy as np
import numpy.typing as npt
from pcgsepy.common.vecs import Orientation, Vec, orientation_from_vec
from pcgsepy.structure import Block, Structure
from typing import List, Optional, Tuple

class HullBuilder:
    def __init__(self,
                 erosion_type: str,
                 apply_erosion: bool,
                 apply_smoothing: bool):
        self.AIR_BLOCK_VALUE = 0
        self.BASE_BLOCK_VALUE = 1
        self.SLOPE_BLOCK_VALUE = 2
        self.CORNER_BLOCK_VALUE = 3
        self.CORNERINV_BLOCK_VALUE = 4
        self.available_erosion_types = ['grey', 'bin']
        self.erosion_type = erosion_type
        assert self.erosion_type in self.available_erosion_types, f'Unrecognized erosion type {self.erosion_type}; available are {self.available_erosion_types}.'
        if self.erosion_type == 'grey':
            self.erosion = grey_erosion
            self.footprint=[
                [
                    [False, False, False],
                    [False, True, False],
                    [False, False, False]
                ],
                [
                    [False, True, False],
                    [True, True, True],
                    [False, True, False]
                ],
                [
                    [False, False, False],
                    [False, True, False],
                    [False, False, False]
                ]
            ]
        elif self.erosion_type == 'bin':
            self.erosion = binary_erosion
            self.iterations = 2
        self.apply_erosion = apply_erosion
        self.apply_smoothing = apply_smoothing
        
        self.base_block = 'MyObjectBuilder_CubeBlock_LargeBlockArmorBlock'
        self.window_block = 'MyObjectBuilder_CubeBlock_Window1x1'
        self._blocks_set = {}
    
    def _get_convex_hull(self,
                         arr: np.ndarray) -> np.ndarray:
        """Compute the convex hull of the given array.

        Args:
            arr (np.ndarray): The Structure's array.

        Returns:
            np.ndarray: The convex hull.
        """
        points = np.transpose(np.where(arr))
        hull = ConvexHull(points)
        deln = Delaunay(points[hull.vertices])
        idx = np.stack(np.indices(arr.shape), axis=-1)
        out_idx = np.nonzero(deln.find_simplex(idx) + 1)
        out_arr = np.zeros(arr.shape)
        out_arr[out_idx] = self.BASE_BLOCK_VALUE
        # out_arr[np.nonzero(arr)] = arr[np.nonzero(arr)]
        return out_arr
        
    def _adj_to_spaceship(self,
                          i: int,
                          j: int,
                          k: int,
                          spaceship: np.ndarray) -> bool:
        """Check coordinates adjacency to original spaceship hull.

        Args:
            i (int): The i coordinate
            j (int): The j coordiante
            k (int): The k coordinate
            spaceship (np.ndarray): The original spaceship hull

        Returns:
            bool: Whether the coordinate is adjacent to the original spaceship
        """
        adj = False
        for di, dj, dk in zip([+1, 0, 0, 0, 0, -1], [0, +1, 0, 0, -1, 0], [0, 0, +1, -1, 0, 0]):
            if 0 < i + di < spaceship.shape[0] and 0 < j + dj < spaceship.shape[1] and 0 < k + dk < spaceship.shape[2]:
                adj |= spaceship[i + di, j + dj, k + dk] != 0
        return adj

    def _add_block(self,
                   block_type: str,
                   idx: Tuple[int, int, int],
                   pos: Vec,
                   orientation_forward: Orientation = Orientation.FORWARD,
                   orientation_up: Orientation = Orientation.UP) -> None:
        """Add the block to the structure.

        Args:
            block_type (str): The block type.
            structure (Structure): The structure.
            pos (Tuple[int, int, int]): The grid coordinates (non-grid-size specific)
            orientation_forward (Orientation, optional): The forward orientation of the block. Defaults to Orientation.FORWARD.
            orientation_up (Orientation, optional): The up orientation of the block. Defaults to Orientation.UP.
        """
        block = Block(block_type=block_type,
                      orientation_forward=orientation_forward,
                      orientation_up=orientation_up)
        block.position = pos
        self._blocks_set[idx] = block

    def _tag_internal_air_blocks(self,
                                 arr: np.ndarray):
        air_blocks = np.zeros(shape=arr.shape)
        for i in range(arr.shape[0]):
            for j in range(arr.shape[1]):
                for k in range(arr.shape[2]):
                    if sum(arr[0:i, j, k]) != 0 and \
                        sum(arr[i:arr.shape[0], j, k]) != 0 and \
                        sum(arr[i, 0:j, k]) != 0 and \
                        sum(arr[i, j:arr.shape[1], k]) != 0 and \
                        sum(arr[i, j, 0:k]) != 0 and \
                        sum(arr[i, j, k:arr.shape[2]]) != 0:
                            air_blocks[i, j, k] = self.BASE_BLOCK_VALUE
        return air_blocks
        
    def _exists_block(self,
                      idx: Tuple[int, int, int],
                      structure: Structure) -> bool:
        return structure._blocks.get(idx, None) is not None
    
    def _within_hull(self,
                     loc: Vec,
                     hull: npt.NDArray[np.float32]) -> bool:
        i, j, k = loc.as_tuple()
        return 0 <= i < hull.shape[0] and 0 <= j < hull.shape[1] and 0 <= k < hull.shape[2]
    
    def _is_valid_block(self,
                        loc: Vec,
                        structure: Structure,
                        hull: np.typing.NDArray) -> bool:
        return (self._exists_block(idx=vec_to_idx(v=loc), structure=structure) and not is_slope_block(structure._blocks.get(vec_to_idx(v=loc), None))) or\
            (self._within_hull(loc=loc.scale(1 / structure.grid_size).to_veci(), hull=hull) and hull[vec_to_idx(v=loc.scale(1 / structure.grid_size).to_veci())] == self.BASE_BLOCK_VALUE)
    
    def _is_air_block(self,
                      loc: Vec,
                      structure: Structure,
                      hull: np.typing.NDArray) -> bool:
        return not self._exists_block(idx=vec_to_idx(v=loc), structure=structure) and\
            (self._within_hull(loc=loc.scale(1 / structure.grid_size).to_veci(), hull=hull) and hull[vec_to_idx(v=loc.scale(1 / structure.grid_size).to_veci())] == self.AIR_BLOCK_VALUE)
    
    def _pointing_against(self,
                          loc: Vec,
                          structure: Structure,
                          direction: Vec) -> bool:
        direction = direction.scale(v=1 / structure.grid_size)
        # print(f'Observing block at {loc} via {direction}:')
        if self._exists_block(vec_to_idx(v=loc), structure=structure):
            obs_block = structure._blocks.get(vec_to_idx(v=loc))
        else:
            obs_block = self._blocks_set.get(vec_to_idx(v=loc.scale(1 / structure.grid_size).to_veci()))
            if obs_block is None:
                return False
        # return obs_block.orientation_forward == direction.opposite() or obs_block.orientation_up == direction.opposite()            
        return obs_block.orientation_up == direction.opposite()
    
    def get_structure_iterators(self,
                                structure: Structure) -> Tuple[Vec, List[Vec]]:
        scale = structure.grid_size
        
        dd, du = Orientation.DOWN.value.scale(v=scale), Orientation.UP.value.scale(v=scale)
        dr, dl = Orientation.RIGHT.value.scale(v=scale), Orientation.LEFT.value.scale(v=scale)
        df, db = Orientation.FORWARD.value.scale(v=scale), Orientation.BACKWARD.value.scale(v=scale)
        
        return scale, [dd, du, dr, dl, df, db]
    
    def _next_to_window(self,
                        loc: Vec,
                        structure: Structure,
                        direction: Vec) -> bool:
        dloc = loc.sum(direction)
        if self._exists_block(vec_to_idx(v=dloc), structure=structure):
            obs_block = structure._blocks.get(vec_to_idx(v=dloc))
            return obs_block.block_type.startswith(self.window_block)
    
    def _remove_in_direction(self,
                             loc: Vec,
                             hull: npt.NDArray[np.float32],
                             direction: Vec) -> npt.NDArray[np.float32]:
        i, j, k = loc.as_tuple()
        di, dj, dk = direction.as_tuple()
        while 0 < i < hull.shape[0] and 0 < j < hull.shape[1] and 0 < k < hull.shape[2]:
            hull[i, j, k] = self.AIR_BLOCK_VALUE
            i += di
            j += dj
            k += dk
        return hull
    
    def _remove_obstructing_blocks(self,
                                   hull: npt.NDArray[np.float32],
                                   structure: Structure) -> npt.NDArray[np.float32]:
        ii, jj, kk = hull.shape
        for i in range(ii):
            for j in range(jj):
                for k in range(kk):
                    if hull[i, j, k] != self.AIR_BLOCK_VALUE:
                        scale, directions = self.get_structure_iterators(structure=structure)
                        loc = idx_to_vec(idx=(scale * i, scale * j, scale * k))
                        for direction in directions:                            
                            if self._next_to_window(loc=loc,
                                                    structure=structure,
                                                    direction=direction):
                                hull = self._remove_in_direction(loc=loc.scale(v=1 / structure.grid_size).to_veci(),
                                                                 hull=hull,
                                                                 direction=direction.opposite().scale(v=1 / structure.grid_size).to_veci())
        return hull
    
    def _slope_replacement(self,
                           loc: Vec,
                           structure: Structure,
                           hull: npt.NDArray[np.float32],
                           directions: List[Vec]) -> bool:
        d1, d2, d3, d4 = directions
        return self._is_valid_block(loc=loc.sum(d1), structure=structure, hull=hull) and \
                self._is_valid_block(loc=loc.sum(d2), structure=structure, hull=hull) and \
                    self._is_air_block(loc=loc.sum(d3), structure=structure, hull=hull) and \
                    self._is_air_block(loc=loc.sum(d4), structure=structure, hull=hull)
    
    def get_directions_combinations(self,
                                    directions: List[Vec],
                                    n: int) -> List[List[Vec]]:
        return combinations(directions, n)
    
    def simple_conversion(self,
                          idx: Tuple[int, int, int],
                          hull: np.typing.NDArray,
                          structure: Structure) -> Optional[Block]:
        i, j, k = idx
        scale, directions = self.get_structure_iterators(structure=structure)
        loc = idx_to_vec(idx=(scale * i, scale * j, scale * k))
         
        # debugging
        print(f'\n\nBlock at {i} {j}, {k}:')
        
        # print(f' {hull[i,j+1,k+1]} \n{hull[i-1,j,k+1]} {hull[i+1,j,k+1]}\n {hull[i,j-1,k+1]}')
        # print(f' {hull[i,j+1,k]} \n{hull[i-1,j,k]} {hull[i+1,j,k]}\n {hull[i,j-1,k]}')
        # print(f' {hull[i,j+1,k-1]} \n{hull[i-1,j,k-1]} {hull[i+1,j,k-1]}\n {hull[i,j-1,k-1]}')
        
        # removal check
        # slopes checks
        for direction in directions:
            # print(loc, direction, loc.sum(direction).scale(1 / structure.grid_size).to_veci())
            if not self._is_valid_block(loc=loc.sum(direction), structure=structure, hull=hull) and \
                self._pointing_against(loc=loc.sum(direction), structure=structure, direction=direction):
                    return None, self.AIR_BLOCK_VALUE
        
        dd, du, dr, dl, df, db = directions
        
        # case-based slope assignment
        if hull[i, j, k] == self.BASE_BLOCK_VALUE:
            for d1, d2, d3, d4 in zip([dd, dd, du, du, df, df, df, df, db, db, db, db],
                                      [dl, dr, dl, dr, du, dd, dr, dl, du, dd, dr, dl],
                                      [du, du, dd, dd, db, db, db, db, df, df, df, df],
                                      [dr, dl, dr, dl, dd, du, dl, dr, dd, du, dl, dr]):
                print(i, j, k, d1, d2, d3, d4)
                if self._slope_replacement(loc=loc,
                                           structure=structure,
                                           hull=hull,
                                           directions=[d1, d2, d3, d4]):
                    print(d1, d1.normalize().to_veci(), orientation_from_vec(d1.normalize().to_veci()))
                    print(d4, d4.normalize().to_veci(), orientation_from_vec(d4.normalize().to_veci()))
                    return Block(block_type='MyObjectBuilder_CubeBlock_LargeBlockArmorSlope',
                                 orientation_forward=orientation_from_vec(d1.normalize().to_veci()),
                                 orientation_up=orientation_from_vec(d4.normalize().to_veci())), self.SLOPE_BLOCK_VALUE

        return None, hull[i, j, k]
        
    def add_external_hull(self,
                          structure: Structure) -> None:
        """Add an external hull to the given Structure.
        This process adds the hull blocks directly into the Structure, so it can be used only once per spaceship.

        Args:
            structure (Structure): The spaceship.
        """
        arr = structure.as_grid_array()
        air = self._tag_internal_air_blocks(arr=arr)
        hull = self._get_convex_hull(arr=arr)
        hull[np.nonzero(air)] = self.AIR_BLOCK_VALUE
        hull[np.nonzero(arr)] = self.AIR_BLOCK_VALUE
        
        
        if self.apply_erosion:
            if self.erosion_type == 'grey':
                hull = grey_erosion(input=hull,
                                    footprint=self.footprint,
                                    mode='constant',
                                    cval=1)
                hull = hull.astype(int)
                hull *= self.BASE_BLOCK_VALUE                
            elif self.erosion_type == 'bin':
                mask = np.zeros(arr.shape)
                for i in range(mask.shape[0]):
                    for j in range(mask.shape[1]):
                        for k in range(mask.shape[2]):
                            mask[i, j, k] = self.AIR_BLOCK_VALUE if self._adj_to_spaceship(i=i, j=j, k=k, spaceship=arr) else self.BASE_BLOCK_VALUE
                hull = binary_erosion(input=hull,
                                      mask=mask,
                                      iterations=self.iterations)
                hull = hull.astype(int)
                hull *= self.BASE_BLOCK_VALUE
        
        # remove all blocks that obstruct windows
        hull = self._remove_obstructing_blocks(hull=hull,
                                               structure=structure)
        
        # add blocks to self._blocks_set
        for i in range(hull.shape[0]):
                for j in range(hull.shape[1]):
                    for k in range(hull.shape[2]):
                        if hull[i, j, k] != self.AIR_BLOCK_VALUE:
                            self._add_block(block_type=self.base_block,
                                            idx=(i, j, k),
                                            pos=Vec.v3i(i, j, k).scale(v=structure.grid_size),
                                            orientation_forward=Orientation.FORWARD,
                                            orientation_up=Orientation.UP)
        
        if self.apply_smoothing:
            modified = 1
            while modified != 0:
                modified = 0
                to_rem = []
                for (i, j, k), block in self._blocks_set.items():
                    print((i, j, k), block)
                    substitute_block, val = self.simple_conversion(idx=(i, j, k),
                                                                   hull=hull,
                                                                   structure=structure)
                    print('->', substitute_block, val)
                    if substitute_block is not None:
                        substitute_block.position = block.position
                        self._blocks_set[(i, j, k)] = substitute_block
                        modified += 1
                    elif substitute_block is None and val == self.AIR_BLOCK_VALUE:
                        to_rem.append((i, j, k))
                        modified += 1
                    hull[i, j, k] = val
                for r in to_rem:
                    self._blocks_set.pop(r)
                # break
        
        # add blocks to structure
        for k, block in self._blocks_set.items():
            structure.add_block(block=block,
                                grid_position=block.position.as_tuple())

In [22]:
hullbuilder = HullBuilder(erosion_type='bin', apply_erosion=True, apply_smoothing=True)

hullbuilder.add_external_hull(structure=structure)

interactive_plot(structure=structure,
                 title='Structure w/ hull')

(1, 8, 1) MyObjectBuilder_CubeBlock_LargeBlockArmorBlock at {'X': 5, 'Y': 40, 'Z': 5}; OF {'X': 0, 'Y': 0, 'Z': -1}; OU {'X': 0, 'Y': 1, 'Z': 0}


Block at 1 8, 1:
-> None 0
(1, 8, 2) MyObjectBuilder_CubeBlock_LargeBlockArmorBlock at {'X': 5, 'Y': 40, 'Z': 10}; OF {'X': 0, 'Y': 0, 'Z': -1}; OU {'X': 0, 'Y': 1, 'Z': 0}


Block at 1 8, 2:
1 8 2 {'X': 0, 'Y': -5, 'Z': 0} {'X': -5, 'Y': 0, 'Z': 0} {'X': 0, 'Y': 5, 'Z': 0} {'X': 5, 'Y': 0, 'Z': 0}
1 8 2 {'X': 0, 'Y': -5, 'Z': 0} {'X': 5, 'Y': 0, 'Z': 0} {'X': 0, 'Y': 5, 'Z': 0} {'X': -5, 'Y': 0, 'Z': 0}
1 8 2 {'X': 0, 'Y': 5, 'Z': 0} {'X': -5, 'Y': 0, 'Z': 0} {'X': 0, 'Y': -5, 'Z': 0} {'X': 5, 'Y': 0, 'Z': 0}
1 8 2 {'X': 0, 'Y': 5, 'Z': 0} {'X': 5, 'Y': 0, 'Z': 0} {'X': 0, 'Y': -5, 'Z': 0} {'X': -5, 'Y': 0, 'Z': 0}
{'X': 0, 'Y': 5, 'Z': 0} {'X': 0, 'Y': 1, 'Z': 0} Orientation.UP
{'X': -5, 'Y': 0, 'Z': 0} {'X': -1, 'Y': 0, 'Z': 0} Orientation.LEFT
-> MyObjectBuilder_CubeBlock_LargeBlockArmorSlope at {'X': 0.0, 'Y': 0.0, 'Z': 0.0}; OF {'X': 0

## In-game placement

In [23]:
place_ingame = True

In [24]:
if place_ingame:
    base_position, orientation_forward, orientation_up = get_base_values()
    # place_structure(structure=structure,
    #                 position=base_position,
    #                 orientation_forward=orientation_forward,
    #                 orientation_up=orientation_up,
    #                 batchify=False)
    structure.update(
        origin=Vec.v3f(0., 0., 100.),
        # orientation_forward=orientation_forward,
        orientation_forward=Orientation.FORWARD,
        # orientation_up=orientation_up,
        orientation_up=Orientation.UP,
    )
    toggle_gamemode(GameMode.PLACING)
    all_blocks = structure.get_all_blocks(to_place=True)
    for block in all_blocks:
        block.position = block.position.sum(base_position)
    place_blocks(all_blocks, sequential=False)
    toggle_gamemode(GameMode.EVALUATING)

In [25]:
# from itertools import product

# orientations = [Orientation.FORWARD, Orientation.BACKWARD, Orientation.UP, Orientation.DOWN, Orientation.LEFT, Orientation.RIGHT]

# base_position, orientation_forward, orientation_up = get_base_values()

# for i, (of, ou) in enumerate(product(orientations, orientations)):
    
#     print(f'{i}: of: {of.name}; ou: {ou.name}')

#     b1 = Block(block_type='MyObjectBuilder_CubeBlock_LargeBlockArmorSlope',
#             orientation_forward=of,
#             orientation_up=ou)
#     b1.position = Vec.v3i(x=(i * 5), y=0, z=0)


#     b1.position = b1.position.sum(base_position)

#     toggle_gamemode(GameMode.PLACING)
#     place_blocks([b1], sequential=False)
#     toggle_gamemode(GameMode.EVALUATING)


In [26]:
# base_position, orientation_forward, orientation_up = get_base_values()

# dd, du = Orientation.DOWN, Orientation.UP
# dr, dl = Orientation.RIGHT, Orientation.LEFT
# df, db = Orientation.FORWARD, Orientation.BACKWARD

# for i, (d1, d2, d3, d4) in enumerate(zip([dd, dd, du, du, df, df, df, df, db, db, db, db],
#                                          [dl, dr, dl, dr, du, dd, dr, dl, du, dd, dr, dl],
#                                          [du, du, dd, dd, db, db, db, db, df, df, df, df],
#                                          [dr, dl, dr, dl, dd, du, dl, dr, dd, du, dl, dr])):
#     b1 = Block(block_type='MyObjectBuilder_CubeBlock_LargeBlockArmorBlock',
#                orientation_forward=Orientation.FORWARD,
#                orientation_up=Orientation.UP)
#     b1.position = Vec.v3f(20 + i * 10, 0, 0).sum(base_position).sum(d3.value.scale(5))
#     b2 = Block(block_type='MyObjectBuilder_CubeBlock_LargeBlockArmorBlock',
#                orientation_forward=Orientation.FORWARD,
#                orientation_up=Orientation.UP)
#     b2.position = Vec.v3f(20 + i * 10, 0, 0).sum(base_position).sum(d4.value.scale(5))
    
#     slope = Block(block_type='MyObjectBuilder_CubeBlock_LargeBlockArmorSlope',
#                   orientation_forward=d1,
#                   orientation_up=d4)
#     slope.position = Vec.v3f(20 + i * 10, 0, 0).sum(base_position)
    
#     print(b1, slope, b2)

#     toggle_gamemode(GameMode.PLACING)
#     place_blocks([b1, slope, b2], sequential=False)
#     toggle_gamemode(GameMode.EVALUATING)